## Imports

In [152]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

Random circuits: https://docs.quantum.ibm.com/api/qiskit/circuit_random#random-circuits

In [153]:
#make a random quantum circuit with qiskit

from qiskit.circuit.random import random_clifford_circuit
 
qiskit_circ = random_clifford_circuit(num_qubits=2, num_gates=100)
qiskit_circ.draw()

┌───┐┌───┐           ┌──────┐┌────┐          ┌──────┐   ┌─────┐┌──────┐»
q_0: ┤ I ├┤ S ├───────────┤0     ├┤ √X ├──■───────┤0     ├─■─┤ Sdg ├┤1     ├»
     ├───┤├───┤┌───┐┌────┐│  Dcx │└────┘┌─┴─┐┌───┐│  Dcx │ │ └─────┘│  Ecr │»
q_1: ┤ X ├┤ Z ├┤ X ├┤ √X ├┤1     ├──────┤ X ├┤ Z ├┤1     ├─■────────┤0     ├»
     └───┘└───┘└───┘└────┘└──────┘      └───┘└───┘└──────┘          └──────┘»
«     ┌──────┐     ┌───┐┌───┐┌────┐┌─────┐     ┌──────┐┌───┐   ┌───┐   ┌───┐»
«q_0: ┤0     ├─────┤ X ├┤ Y ├┤ √X ├┤ Sdg ├──■──┤ √Xdg ├┤ Y ├─X─┤ Z ├─■─┤ S ├»
«     │  Dcx │┌───┐└─┬─┘├───┤└────┘└─────┘┌─┴─┐└┬───┬─┘└─┬─┘ │ ├───┤ │ └───┘»
«q_1: ┤1     ├┤ H ├──■──┤ Z ├─────────────┤ Y ├─┤ Y ├────■───X─┤ Z ├─■──────»
«     └──────┘└───┘     └───┘             └───┘ └───┘          └───┘        »
«          ┌──────┐┌────┐┌───┐┌────────┐┌─────┐┌───┐     ┌───┐┌─────┐     »
«q_0: ──■──┤0     ├┤ √X ├┤ Z ├┤0       ├┤ Sdg ├┤ Y ├──■──┤ S ├┤ Sdg ├──■──»
«     ┌─┴─┐│  Dcx │└────┘└───┘│  Iswap │└─────┘└───┘┌─┴─┐└───┘└─────┘┌─┴─┐»
«q_1: ┤ Y ├┤1     ├───────────┤1       ├────────────┤ Y ├────────────┤ Y ├»
«     └───┘└──────┘           └────────┘            └───┘            └───┘»
«     ┌──────┐          ┌──────┐      ┌────┐┌────┐   ┌──────┐┌─────┐┌───┐»
«q_0: ┤ √Xdg ├──────────┤0     ├──■───┤ √X ├┤ √X ├─■─┤ √Xdg ├┤ Sdg ├┤ X ├»
«     └┬───┬─┘┌───┐┌───┐│  Dcx │┌─┴─┐┌┴────┤└────┘ │ └┬───┬─┘└┬───┬┘└───┘»
«q_1: ─┤ S ├──┤ Z ├┤ S ├┤1     ├┤ X ├┤ Sdg ├───────■──┤ Y ├───┤ H ├──────»
«      └───┘  └───┘└───┘└──────┘└───┘└─────┘          └───┘   └───┘      »
«     ┌──────┐          ┌───┐   ┌──────┐┌──────┐           ┌───┐┌───┐        »
«q_0: ┤0     ├───────■──┤ I ├─X─┤0     ├┤1     ├──■───■──X─┤ Y ├┤ I ├──■───■─»
«     │  Dcx │┌───┐┌─┴─┐└───┘ │ │  Ecr ││  Dcx │┌─┴─┐ │  │ └─┬─┘└───┘┌─┴─┐ │ »
«q_1: ┤1     ├┤ Z ├┤ Y ├──────X─┤1     ├┤0     ├┤ X ├─■──X───■───────┤ X ├─■─»
«     └──────┘└───┘└───┘        └──────┘└──────┘└───┘                └───┘   »
«     ┌──────┐        ┌─────┐┌──────┐┌──────┐┌────────┐┌───┐        ┌─────┐»
«q_0: ┤1     ├──────X─┤ Sdg ├┤0     ├┤ √Xdg ├┤0       ├┤ I ├──────■─┤ Sdg ├»
«     │  Ecr │┌───┐ │ └─────┘│  Dcx │└┬────┬┘│  Iswap │├───┤┌───┐ │ └┬───┬┘»
«q_1: ┤0     ├┤ S ├─X────────┤1     ├─┤ √X ├─┤1       ├┤ S ├┤ X ├─■──┤ H ├─»
«     └──────┘└───┘          └──────┘ └────┘ └────────┘└───┘└───┘    └───┘ »
«      ┌───┐  ┌───┐    ┌───┐    ┌──────┐     ┌────────┐┌───┐           ┌───┐»
«q_0: ─┤ I ├──┤ X ├─■──┤ Y ├──■─┤0     ├─────┤0       ├┤ Z ├───────────┤ X ├»
«     ┌┴───┴─┐└─┬─┘ │ ┌┴───┴┐ │ │  Dcx │┌───┐│  Iswap │├───┴┐┌───┐┌───┐└─┬─┘»
«q_1: ┤ √Xdg ├──■───■─┤ Sdg ├─■─┤1     ├┤ I ├┤1       ├┤ √X ├┤ Z ├┤ S ├──■──»
«     └──────┘        └─────┘   └──────┘└───┘└────────┘└────┘└───┘└───┘     »
«     ┌───┐
«q_0: ┤ X ├
«     └───┘
«q_1: ─────
«

Convert to qasm via Qiskit: https://docs.quantum.ibm.com/api/qiskit/qasm2#exporting-examples

In [154]:
import qiskit.qasm2
from qiskit.circuit import QuantumCircuit
 
print(qiskit.qasm2.dumps(qiskit_circ))

OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 { cx q0,q1; cx q1,q0; }
gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }
gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
qreg q[2];
x q[1];
id q[0];
z q[1];
x q[1];
sx q[1];
s q[0];
dcx q[0],q[1];
sx q[0];
cx q[0],q[1];
z q[1];
dcx q[0],q[1];
cz q[1],q[0];
sdg q[0];
ecr q[1],q[0];
dcx q[0],q[1];
h q[1];
cx q[1],q[0];
y q[0];
sx q[0];
sdg q[0];
z q[1];
cy q[0],q[1];
sxdg q[0];
y q[1];
cy q[1],q[0];
swap q[0],q[1];
z q[0];
z q[1];
cz q[1],q[0];
s q[0];
cy q[0],q[1];
dcx q[0],q[1];
sx q[0];
z q[0];
iswap q[0],q[1];
sdg q[0];
y q[0];
cy q[0],q[1];
s q[0];
sdg q[0];
cy q[0],q[1];
s q[1];
z q[1];
s q[1];
sxdg q[0];
dcx q[0],q[1];
cx q[0],q[1];
sdg q[1];
sx q[0];
sx q[0];
cz q[1],q[0];
y q[1];
sxdg q[0];
sdg q[0];
x q[0];
h q[1];
dcx q[0],q[1];
z q[1];
cy q[0],q[1];
id q[0];
swap q[0],q[1];
ecr q[0],q[1];
dcx q[1],q[0];
cx q[0],q[1];
cz q[1],

In [155]:
#Now add it to a file for clarity

qiskit.qasm2.dump(qiskit_circ, "random_circuit.qasm")
path = "random_circuit.qasm"

## QASM Completed ^

## QASM Conversion 

### Imports

In [156]:
import numpy as np
from qutip import about, basis, rand_ket, tensor
from qutip_qip.operations import Measurement
from qutip_qip.operations.gates import gate_sequence_product
from qutip_qip.qasm import read_qasm, print_qasm

### Read in the file to a string

In [157]:
with open(path, 'r') as file:
    qasm_str = file.read()

In [158]:
type(qasm_str)

str

In [159]:
print(qasm_str)

OPENQASM 2.0;
include "qelib1.inc";
gate dcx q0,q1 { cx q0,q1; cx q1,q0; }
gate rzx(param0) q0,q1 { h q1; cx q0,q1; rz(param0) q1; cx q0,q1; h q1; }
gate ecr q0,q1 { rzx(pi/4) q0,q1; x q0; rzx(-pi/4) q0,q1; }
gate iswap q0,q1 { s q0; s q1; h q0; cx q0,q1; cx q1,q0; h q1; }
qreg q[2];
x q[1];
id q[0];
z q[1];
x q[1];
sx q[1];
s q[0];
dcx q[0],q[1];
sx q[0];
cx q[0],q[1];
z q[1];
dcx q[0],q[1];
cz q[1],q[0];
sdg q[0];
ecr q[1],q[0];
dcx q[0],q[1];
h q[1];
cx q[1],q[0];
y q[0];
sx q[0];
sdg q[0];
z q[1];
cy q[0],q[1];
sxdg q[0];
y q[1];
cy q[1],q[0];
swap q[0],q[1];
z q[0];
z q[1];
cz q[1],q[0];
s q[0];
cy q[0],q[1];
dcx q[0],q[1];
sx q[0];
z q[0];
iswap q[0],q[1];
sdg q[0];
y q[0];
cy q[0],q[1];
s q[0];
sdg q[0];
cy q[0],q[1];
s q[1];
z q[1];
s q[1];
sxdg q[0];
dcx q[0],q[1];
cx q[0],q[1];
sdg q[1];
sx q[0];
sx q[0];
cz q[1],q[0];
y q[1];
sxdg q[0];
sdg q[0];
x q[0];
h q[1];
dcx q[0],q[1];
z q[1];
cy q[0],q[1];
id q[0];
swap q[0],q[1];
ecr q[0],q[1];
dcx q[1],q[0];
cx q[0],q[1];
cz q[1],

In [160]:
from qiskit.circuit.library import SXdgGate

# Decompose the SXdg gate using Qiskit
sxdg_gate = SXdgGate()
decomposition_circuit = sxdg_gate.definition
print(decomposition_circuit.draw())

global phase: 7π/4
   ┌───┐┌───┐┌───┐
q: ┤ S ├┤ H ├┤ S ├
   └───┘└───┘└───┘


### Append basic gate definitions to the string

For more definitions, see this repo: https://github.com/HQSquantumsimulations/qoqo_qasm/blob/aff4aba03ae12e12599ed11b98c9b7ecaef0c021/roqoqo-qasm/src/interface.rs#L1345

In [161]:
# Insert gate definitions after the 'include' line in qasm_str

swap_gate_def = [
    'gate swap a, b{',
    'cx b, a;',
    'cx a, b;',
    'cx b, a;',
    '}'
]

sx_gate_def = [
    'gate sx a {',
    'u1(-pi/2) a;',
    'u2(0,pi) a;',
    'u1(-pi/2) a;',
    '}'
]

sxdg_gate_def = [
    'gate sxdg a {',
    'u1(pi/2) a;',
    'u2(0,pi) a;',
    'u1(pi/2) a;',
    '}'
]

lines = qasm_str.splitlines()
for i, line in enumerate(lines):
    if line.strip().startswith('include'):
        insert_idx = i + 1
        break

gate_defs = swap_gate_def + [''] + sx_gate_def + [''] + sxdg_gate_def + ['']
lines[insert_idx:insert_idx] = gate_defs
qasm_str = '\n'.join(lines)

### Review the string using the built-in qasm to qutip conversion tools

In [162]:
qutip_circ = read_qasm(qasm_str, strmode=True)
gate_sequence_product(qutip_circ.propagators())

Quantum object: dims=[[2, 2], [2, 2]], shape=(4, 4), type='oper', dtype=CSR, isherm=False
Qobj data =
[[-0.5+0.5j  0. +0.j   0. +0.j   0.5-0.5j]
 [ 0. +0.j   0.5+0.5j -0.5-0.5j  0. +0.j ]
 [-0.5+0.5j  0. +0.j   0. +0.j  -0.5+0.5j]
 [ 0. +0.j  -0.5-0.5j -0.5-0.5j  0. +0.j ]]